<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_DIY02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# Faça você mesmo 02.

## 1. Crie um banco de dados no DW do Spark chamado VendasVarejo, e persista todas as tabelas neste banco de dados.

In [15]:
# Carregando todos os dados no formato DataFrame

clientes = spark.read.load(('/content/drive/MyDrive/Datasets/pyspark_course/Ati'
                            'vidades/Clientes.parquet'), format="parquet")

itens_vendas = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course"
                                "/Atividades/ItensVendas.parquet"), 
                                format="parquet")

produtos = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/Ati"
                            "vidades/Produtos.parquet"), format="parquet")

vendas = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/Ativ"
                          "idades/Vendas.parquet"), format="parquet")

vendedores = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/A"
                            "tividades/Vendedores.parquet"), format="parquet")

dataframes = [clientes, itens_vendas, produtos, vendas, vendedores]     
frame_names = ['Clientes', 'ItensVendas', 'Produtos', 'Vendas', 'Vendedores']
for frame, name in zip(dataframes,frame_names):
  print(name)
  frame.show(5)

Clientes
+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
+---------+--------------------+------+------+--------+
only showing top 5 rows

ItensVendas
+---------+--------+----------+-------------+----------+--------+----------------+
|ProdutoID|VendasID|Quantidade|ValorUnitario|ValorTotal|Desconto|TotalComDesconto|
+---------+--------+----------+-------------+----------+--------+----------------+
|        2|     400|         2|       9201.0|   18402.0| 6256,68|        12145.32|
|        2|     385|         2|       9201.0|   18402.0| 5704,62|        12697.38|
|        4|     395|         2|    

- Criando o banco de dados VendasVarejo

In [11]:
spark.sql('Create database if not exists VendasVarejo')

spark.sql('show databases').show()

+------------+
|   namespace|
+------------+
|     default|
|vendasvarejo|
+------------+



In [12]:
spark.sql('use vendasvarejo').show()

++
||
++
++



- Escrevendo todos os DataFrames como tabelas usando um loop.

In [13]:
for frame, name in zip(dataframes, frame_names):
  frame.write.saveAsTable(f'{name}')

In [14]:
spark.sql('show tables').show()

+------------+-----------+-----------+
|   namespace|  tableName|isTemporary|
+------------+-----------+-----------+
|vendasvarejo|   clientes|      false|
|vendasvarejo|itensvendas|      false|
|vendasvarejo|   produtos|      false|
|vendasvarejo|     vendas|      false|
|vendasvarejo| vendedores|      false|
+------------+-----------+-----------+



## 2. Crie uma consulta que mostre de cada item vendido: Nome do Cliente, Data da Venda, Produto, Vendedor e Valor Total do item.

In [18]:
spark.sql(('select c.Cliente, v.Data, p.Produto, ve.Vendedor, iv.ValorTotal'
' from itensvendas as iv inner join vendas as v on iv.VendasID = v.VendasID '
'inner join vendedores as ve on v.VendedorID = ve.VendedorID inner join '
'clientes as c on v.ClienteID = c.ClienteID inner join produtos as p on'
' iv.ProdutoID = p.ProdutoID')).show()

+-----------------+--------+--------------------+----------------+----------+
|          Cliente|    Data|             Produto|        Vendedor|ValorTotal|
+-----------------+--------+--------------------+----------------+----------+
|   Cosme Zambujal|1/1/2019|Bicicleta Altools...|    Armando Lago|   7820.85|
|   Cosme Zambujal|1/1/2019|Bermuda Predactor...|    Armando Lago|     97.75|
|   Cosme Zambujal|1/1/2019|Camiseta Predacto...|    Armando Lago|     135.0|
|Gertrudes Hidalgo|1/1/2020|Luva De Ciclismo ...|   Iberê Lacerda|     150.4|
| Antão Corte-Real|2/1/2020|Capacete Gometws ...|Jéssica Castelão|     155.0|
| Antão Corte-Real|2/1/2020|Bicicleta Gometws...|Jéssica Castelão|    5932.0|
| Antão Corte-Real|2/1/2019|Bicicleta Altools...|  Hélio Liberato|   7820.85|
| Antão Corte-Real|2/1/2019|Bermuda Predactor...|  Hélio Liberato|     97.75|
| Antão Corte-Real|2/1/2019|Bicicleta Gometws...|  Hélio Liberato|    5910.0|
| Antão Corte-Real|3/1/2018|Bicicleta Gometws...|  Hélio Liberat